In [ ]:
# | default_exp transforms/spatial

# Imports

In [ ]:
# | export


import torch
from monai.data import MetaTensor
from monai.transforms.spatial.dictionary import RandFlipd, Resized

# Transforms

In [ ]:
# | export


class RandFlipWithCropTrackingd(RandFlipd):
    def __init__(
        self,
        crop_offset_key: str = "crop_offset",
        original_shape_key: str = "original_shape",
        *args,
        **kwargs,
    ) -> MetaTensor:
        super().__init__(*args, **kwargs)
        self.crop_offset_key = crop_offset_key
        self.original_shape_key = original_shape_key

    def __call__(self, data, *args, **kwargs):
        output = super().__call__(data, *args, **kwargs)
        # Check if image was flipped and was cropped previously
        if self._do_transform and self.crop_offset_key in data:
            # Ensure all inputs had input of same size for consistent crop value
            shapes = [data[key].shape for key in self.keys]
            if not all(shape == shapes[0] for shape in shapes):
                raise ValueError("All inputs must have the same shape for consistent crop value")
            crop_shape = shapes[0]
            orig_shape = torch.Tensor(data[self.original_shape_key])
            # New crop value will be flipped
            crop_offset = torch.Tensor(data[self.crop_offset_key])
            # Flip crop value
            crop_offset[-1] = orig_shape[-1] - crop_offset[-1] - crop_shape[-1]
            # Update crop value
            output[self.crop_offset_key] = crop_offset
        return output

In [ ]:
img = torch.rand(4, 100, 100, 100)
x = {"image": img, "crop_offset": (10, 20, 30), "original_shape": (200, 200, 200)}

RandFlipWithCropTrackingd(keys=["image"], prob=1.0)(x)


{
    'image': metatensor([[[[6.8545e-01, 6.0399e-01, 3.4056e-01,  ..., 8.5693e-01,
           3.3220e-01, 7.3319e-01],
          [9.4987e-01, 5.7073e-01, 4.8854e-01,  ..., 3.8559e-01,
           8.2196e-01, 8.8976e-01],
          [1.6620e-01, 2.8657e-01, 1.0281e-01,  ..., 3.9847e-01,
           4.7287e-01, 6.4543e-01],
          ...,
          [2.5086e-01, 6.9413e-01, 3.3679e-02,  ..., 9.3587e-01,
           1.2995e-01, 7.0720e-01],
          [5.8775e-01, 3.6922e-01, 9.1223e-01,  ..., 7.4780e-02,
           8.7514e-01, 7.2446e-01],
          [7.4670e-01, 6.9396e-01, 4.8773e-01,  ..., 1.9586e-01,
           1.0285e-01, 9.2039e-01]],

         [[2.2635e-01, 1.2909e-01, 7.6754e-01,  ..., 8.4512e-01,
           8.0762e-01, 8.9228e-01],
          [6.3988e-01, 5.9801e-01, 4.0705e-01,  ..., 3.4378e-01,
           9.6809e-01, 5.7935e-01],
          [3.1901e-01, 8.1188e-01, 9.3237e-01,  ..., 5.2693e-01,
           8.9355e-01, 5.9845e-01],
          ...,
          [9.5754e-01, 9.0539e-01, 8.44

In [ ]:
# | export


class ResizedWithCropTrackingd(Resized):
    def __init__(
        self,
        crop_offset_key: str = "crop_offset",
        original_shape_key: str = "original_shape",
        *args,
        **kwargs,
    ) -> MetaTensor:
        super().__init__(*args, **kwargs)
        self.crop_offset_key = crop_offset_key
        self.original_shape_key = original_shape_key

    def __call__(self, data, *args, **kwargs):
        # Check if image was cropped previously
        if self.crop_offset_key in data:
            # Ensure all inputs had input of same size for consistent crop value
            shapes = [data[key].shape for key in self.keys]
            if not all(shape == shapes[0] for shape in shapes):
                raise ValueError("All inputs must have the same shape for consistent crop value")
            old_shape = data[self.keys[0]].shape[1:]

        output = super().__call__(data, *args, **kwargs)

        # Check if image was cropped previously
        if self.crop_offset_key in data:
            # Calculate scale of resizing along every axis
            new_shape = output[self.keys[0]].shape[1:]
            scale = torch.tensor(new_shape) / torch.tensor(old_shape)

            # Update crop_offset
            crop_offset = torch.Tensor(data[self.crop_offset_key])
            output[self.crop_offset_key] = (crop_offset * scale).int()

            # Update original_shape
            if self.original_shape_key in data:
                original_shape = torch.Tensor(data[self.original_shape_key])
                output[self.original_shape_key] = (original_shape * scale).int()

        return output

In [ ]:
img = torch.rand(4, 100, 100, 100)
x = {"image": img, "crop_offset": (10, 20, 30), "original_shape": (200, 200, 200)}

ResizedWithCropTrackingd(keys=["image"], spatial_size=(60, 60, 60))(x)


{
    'image': metatensor([[[[0.3897, 0.4213, 0.4188,  ..., 0.5719, 0.3877, 0.2938],
          [0.5080, 0.3272, 0.3928,  ..., 0.5251, 0.4607, 0.3944],
          [0.5273, 0.4151, 0.4860,  ..., 0.2980, 0.4643, 0.4867],
          ...,
          [0.3354, 0.4427, 0.4709,  ..., 0.6162, 0.5052, 0.5085],
          [0.3427, 0.4864, 0.5543,  ..., 0.5740, 0.5100, 0.5218],
          [0.4705, 0.5747, 0.7474,  ..., 0.5422, 0.3211, 0.2885]],

         [[0.4726, 0.5219, 0.5796,  ..., 0.4721, 0.5202, 0.5344],
          [0.5473, 0.4960, 0.5958,  ..., 0.4709, 0.4862, 0.4320],
          [0.6134, 0.4451, 0.5854,  ..., 0.4083, 0.4861, 0.4441],
          ...,
          [0.3651, 0.4200, 0.5483,  ..., 0.5350, 0.4844, 0.5764],
          [0.3959, 0.4391, 0.5437,  ..., 0.4367, 0.4691, 0.4318],
          [0.3764, 0.4810, 0.5528,  ..., 0.4353, 0.3051, 0.2930]],

         [[0.6133, 0.5551, 0.5132,  ..., 0.6203, 0.6347, 0.5752],
          [0.5382, 0.4943, 0.5914,  ..., 0.6614, 0.6017, 0.4949],
          [0.4694, 0.2

# nbdev

In [ ]:
!nbdev_export